In [7]:
#!/bin/bash

# Read CSV from Patch
CSV_FILE="/workspaces/ByteMe/ICT4D/CrashCourse1/Assignment/SudanCities.csv"
echo "lat,lon,addresstype,name" > Sudan_nominatim.csv
# Read the CSV line by line, skipping the header
tail -n +2 "$CSV_FILE" | while IFS=, read -r city country lat lng other_fields
do
    # Trim potential whitespace around city and country
    city=$(echo "$city" | xargs)
    country=$(echo "$country" | xargs)

    # URL encode city names that include spaces
    city_encoded=$(echo "$city" | sed 's/ /%20/g')

    # Display search information
    echo "Searching for: City=$city, Country=$country"

    # Use curl to make the API request and parse the output with jq
    curl -s "https://nominatim.openstreetmap.org/search?q=$city_encoded,$country&format=json" | \
    jq '.[] | [.lat, .lon, .addresstype, .name] | join(",")' >> Sudan_nominatim.csv

    # Optional: sleep to avoid hitting the API rate limit
    sleep 1
done


Searching for: City=Khartoum, Country=Sudan
Searching for: City=Omdurman, Country=Sudan
Searching for: City=Al Mijlad, Country=Sudan
Searching for: City=Khartoum North, Country=Sudan
Searching for: City=Port Sudan, Country=Sudan
Searching for: City=El Geneina, Country=Sudan
Searching for: City=Kassala, Country=Sudan
Searching for: City=El Fasher, Country=Sudan
Searching for: City=Nyala, Country=Sudan
Searching for: City=El Obeid, Country=Sudan
Searching for: City=Gedaref, Country=Sudan
Searching for: City=Wad Medani, Country=Sudan
Searching for: City=Kusti, Country=Sudan
Searching for: City=Ed Daein, Country=Sudan
Searching for: City=Singa, Country=Sudan
Searching for: City=Kurmuk, Country=Sudan
Searching for: City=Ar Ruseris, Country=Sudan
Searching for: City=Ed Damazin, Country=Sudan
Searching for: City=Al Manaqil, Country=Sudan
Searching for: City=Sannar, Country=Sudan
Searching for: City=Atbara, Country=Sudan
Searching for: City=Ed Damer, Country=Sudan
Searching for: City=An Nuhud,

In [1]:
import pandas as pd
import requests

# Load the CSV file into a DataFrame
file_path = '/workspaces/ByteMe/ICT4D/CrashCourse1/Assignment/Sudan-nominatim-Openrefine.csv'  # Update the path to your CSV file
data = pd.read_csv(file_path)

# Function to fetch weather data
def fetch_weather_data(lat, lon):
    url = f"https://previous-runs-api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&hourly=precipitation&start_date=2023-01-01&end_date=2023-12-31"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Iterate over DataFrame rows and fetch weather data for each
results = []
for index, row in data.iterrows():
    json_response = fetch_weather_data(row['lat'], row['lon'])
    if json_response:
        # Extract hourly precipitation data and calculate the sum
        precipitation = json_response.get('hourly', {}).get('precipitation', [])
        total_precipitation = sum(precipitation)
        results.append({
            'latitude': row['lat'],
            'longitude': row['lon'],
            'total_precipitation': total_precipitation
        })
    else:
        results.append({
            'latitude': row['lat'],
            'longitude': row['lon'],
            'total_precipitation': 'Failed to fetch data'
        })

# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results)
print(results_df)
# Save results to a CSV file
output_file_path = '/workspaces/ByteMe/ICT4D/CrashCourse1/Assignment/Sudan-OpenMeteo2023.csv'  # Define the output file path
results_df.to_csv(output_file_path, index=False)
print(f"Data saved to {output_file_path}")

     latitude  longitude  total_precipitation
0   15.563597  32.534912                 32.6
1   15.603086  32.526223                 32.6
2   15.644554  32.477731                 32.6
3   11.783333  30.916667                297.3
4   12.416512  34.296468                295.0
..        ...        ...                  ...
80  13.184465  32.739806                123.2
81  11.800000  28.400000                235.0
82  12.949878  31.925051                102.6
83  14.766667  33.633333                 96.4
84  11.710485  28.344898                235.0

[85 rows x 3 columns]
Data saved to /workspaces/ByteMe/ICT4D/CrashCourse1/Assignment/Sudan-OpenMeteo2023.csv


In [11]:
#Merge | Join Sudan files to have a full table with more data to use via Bi or GIS

import pandas as pd

Data1 = pd.read_csv('/workspaces/ByteMe/ICT4D/CrashCourse1/Assignment/Sudan_nominatim.csv')
Data2 = pd.read_csv('/workspaces/ByteMe/ICT4D/CrashCourse1/Assignment/Sudan-OpenMeteo2023.csv')

# Merge the DataFrames on a common column, e.g., 'user_id'
merged_df = pd.merge(Data1, Data2, on=['lat','lon'], how='left')

# Save the merged DataFrame to a new CSV file

merged_df.to_csv('/workspaces/ByteMe/ICT4D/CrashCourse1/Assignment/SudanMerged.csv', index=False)

